#               **Twitter Disaster Challenge**







The main idea of this task it's how we can classify twitter disaster messages.

I've developed several approaches to solve this problem and analyze the data.

I wanted to focus on how we can work with the data and how it will affect the results, because I think this is the interesting part.
Therefore, in my approach, I did not remove hashtags from the text and use neural networks.

First of all, it's features engineering approach: I've analyzed the data and created new features using hashtags and keywords from text messages.

After that, I used embedding approaches (tf-idf and word2vec) to create vectorize.

For both embedding approaches, I've applied SVM, Support Vector Classifier trained with Stochastic Gradient Descent, Naive Bayes, Decision Tree, and Random Forest to classify.

Moreover, I've used precision, recall, score metrics to evaluate the model's results.

## Library implementation and download the data (using GDrive)

In [ ]:
!pip install transformers
!pip install -U fastai

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from PIL import Image
from torch.nn.functional import cosine_similarity
from torchvision import models, transforms
import pandas as pd
import numpy as np
import re


from transformers import AutoTokenizer, AutoModelForSequenceClassification
from fastai.text.all import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score



import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip /content/gdrive/MyDrive/apli_challenge.zip -d apli_challenge

Archive:  /content/gdrive/MyDrive/apli_challenge.zip
replace apli_challenge/APLI CHALLENGE/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
test_path = '/content/apli_challenge/APLI CHALLENGE/test.csv'
train_path = '/content/apli_challenge/APLI CHALLENGE/train.csv'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Data analysis part


In [ ]:
test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)

In [ ]:
train_df.shape, test_df.shape

((7613, 5), (3263, 5))

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [ ]:
train_df['text']

0                                                                           Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1                                                                                                          Forest fire near La Ronge Sask. Canada
2           All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
3                                                                               13,000 people receive #wildfires evacuation orders in California 
4                                                        Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 
                                                                          ...                                                                    
7608                                                          Two giant cranes holding a bridge collapse into nearby homes h

In [ ]:
train_df[~train_df['keyword'].isnull()]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty much all been wrecked hahaha shoutout to my family for that one,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words that wrecked Disney's stock http://t.co/7enNulLKzM,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Lion gig tonight. Hearing is wrecked. http://t.co/oMNBAtJEAO,0


In [ ]:
train_df['keyword'].unique() # look at the unique 'keyword' values

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [ ]:
test_df.head() # we can see that  in 'text' column we have text with hash tag sign('#') and some url's.

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone.",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,1


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
 4   target    3263 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 127.6+ KB


In [ ]:
train_df["target"].value_counts()/train_df.shape[0]

0    0.57034
1    0.42966
Name: target, dtype: float64

## Feature extraction part

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support 

### Hashtag approach

In [ ]:
# get the hashtags set with the hashtags that appear in the text more then "level" times
def find_hashtags(train_df, level=0): 
    train_df['text'] = train_df['text'].apply(lambda x: x.lower()) #map all the text to lowercase
    train_df["tag"] = train_df["text"].apply(lambda x: re.findall(r"#(\w+)", x)) #find all of the hashtags
    tags=set([]) 
    for loc_tags in train_df["tag"]: 
        for tag in loc_tags:
            if not (tag is None):
                train_df[tag]=train_df["text"].apply(lambda x: int("#"+tag in x))
                if train_df[tag].sum()>level:
                    tags.add(tag)
                del train_df[tag]
    del train_df["tag"]
    return tags

#we get extra features for the dataset, that illustrated does the specific hashtag belong to the text
def to_hashtags(train, tags):
    for tag in tags:
        train[tag]=train["text"].apply(lambda x: int("#"+tag in x))
    train["text"]=train["text"].apply(lambda x: x.replace("#", ""))
    del train["text"]
    del train["location"]
    del train["keyword"]
    del train["id"]
    return train

In [ ]:
#now just load data get the hashtags and try to figure can the hashtags alone give the information about markup
tags = find_hashtags(train_df)
train_df = to_hashtags(train_df, tags) 
#right here we get the embedding of 0 and 1 where the text either contains the hashtag or not
test_df = to_hashtags(test_df, tags)
#N.B. note, that we use the same "tags" set for the test sample, since we got to get the embedding
#into the same space. We do not find the tags for the test data.

train_target = train_df["target"]
train_features = train_df.copy() # here we use copy method not to change the "train_df" itself during processing
del train_features["target"]

X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


#### Support Vector Classifier for hashtag

In [ ]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.6014445173998687
(array([0.97142857, 0.10185185]), array([0.59357542, 0.72527473]), array([0.73688773, 0.17861976]), array([1432,   91]))


#### Support Vector Classifier trained with Stochastic Gradient Descent for hashtag

In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.6047275114904793
(array([0.95085714, 0.13734568]), array([0.59813084, 0.67424242]), array([0.73433363, 0.22820513]), array([1391,  132]))


#### Naive Bayes for hashtag

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
model = MultinomialNB()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.6066973079448457
(array([0.95428571, 0.13734568]), array([0.5989957 , 0.68992248]), array([0.73600705, 0.22908623]), array([1394,  129]))


### Keywords approach

In [ ]:
test = pd.read_csv(test_path)
train = pd.read_csv(train_path)

In [ ]:
def find_keywords(train, level=0):
    train["keyword"] = train["keyword"].apply(lambda x: str(x).lower().split(","))
    keywords=set([])
    for loc_keywords in train["keyword"]:
        for keyword in loc_keywords:
            if not (keyword == "nan"):
                train[keyword]=train["keyword"].apply(lambda x: int(keyword in x))
                if train[keyword].sum()>level:
                    keywords.add(keyword)
                del train[keyword]
    return keywords

def to_keywords(train, keywords):
    train["keyword"] = train["keyword"].apply(lambda x: str(x).lower().split(","))
    for keyword in keywords:
        train[keyword]=train["keyword"].apply(lambda x: int(keyword in x))
    del train["text"]
    del train["location"]
    del train["id"]
    del train["keyword"]
    return train

In [ ]:
keywords = find_keywords(train)
train_df = to_keywords(train, keywords)
test_df = to_keywords(test, keywords)



train_target = train_df["target"]
train_features = train_df.copy()
del train_features["target"]

X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  app.launch_new_instance()


#### Support Vector Classifier for keywords

In [ ]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.5633617859487853
(array([1., 0.]), array([0.56336179, 0.        ]), array([0.72070559, 0.        ]), array([1523,    0]))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Support Vector Classifier trained with Stochastic Gradient Descent for keywords

In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.5633617859487853
(array([1., 0.]), array([0.56336179, 0.        ]), array([0.72070559, 0.        ]), array([1523,    0]))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Naive Bayes for keywords

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
model = MultinomialNB()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.5633617859487853
(array([1., 0.]), array([0.56336179, 0.        ]), array([0.72070559, 0.        ]), array([1523,    0]))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


So, we got it, about 0.6. Surely, you can try to concatenate the embeddings of the keywords and hashtags, and even add it to the text embedding, they can get the more accurate classification, but still do not work alone.

## Data processing part

### TF-IDF vectorization

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
test = pd.read_csv(test_path)
train = pd.read_csv(train_path)
train["text"] = train["text"].apply(lambda x: x.lower())

In [ ]:
stopWords = set(stopwords.words('english')) 
for w in stopWords:
    train["text"] = train["text"].apply(lambda x: x.replace(" "+w+" ", " "))

vectorizer = TfidfVectorizer() #use the TF-IDF model for text embedding

train["text"] =train["text"].apply(lambda x: x.replace("#","")) 

tfidf_train = vectorizer.fit_transform(train["text"])

from sklearn.decomposition import PCA

#The problem is that the dimention of the text embedding in TF-IDF model or BOW model is too large, 
#so we got to reduce the dimentionality, thus, we try to make some
#feature as the Principal Component Analysis.

pca = PCA(n_components=500)
lower_dim_tfidf_train = pca.fit_transform(tfidf_train.todense())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
train_df = lower_dim_tfidf_train
train_target = train["target"]
train_features = train_df.copy()

X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.2)

#### SVC

In [ ]:
from sklearn.svm import SVC

model = SVC()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.7957977675640184
(array([0.90437788, 0.6519084 ]), array([0.77492596, 0.8372549 ]), array([0.83466241, 0.73304721]), array([1013,  510]))


#### SGD

In [ ]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.7898883782009193
(array([0.89516129, 0.65038168]), array([0.77236581, 0.82398453]), array([0.82924226, 0.72696246]), array([1006,  517]))


#### Decision Tree



In [ ]:
from sklearn.tree import DecisionTreeClassifier 

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.6979645436638214
(array([0.73041475, 0.65496183]), array([0.7372093 , 0.64705882]), array([0.7337963 , 0.65098634]), array([860, 663]))


#### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier  #here we add Random Forest Classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.778069599474721
(array([0.91359447, 0.59847328]), array([0.75094697, 0.83940043]), array([0.82432432, 0.69875223]), array([1056,  467]))


### Word2Vec embedding

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split

In [ ]:
test = pd.read_csv(test_path)
train = pd.read_csv(train_path)
some_texts = train["text"].apply(lambda x: x.lower()).apply(lambda x: x.replace("#",""))

stopWords = set(stopwords.words('english'))

for w in stopWords:
    some_texts = some_texts.apply(lambda x: x.replace(" "+w+" ", " "))

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(some_texts)]

model = Doc2Vec(documents, vector_size=1000, workers=4, epochs=50)
textdf = model.docvecs

train_target = train["target"]
train_features = textdf

X_train, X_test, y_train, y_test = train_test_split(train_features, train_target, test_size=0.2)

#### SVC

In [ ]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.6710439921208142
(array([0.80280047, 0.5015015 ]), array([0.6745098, 0.6640159]), array([0.73308471, 0.57142857]), array([1020,  503]))


#### SGD

In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.639527248850952
(array([0.65344224, 0.62162162]), array([0.68965517, 0.58227848]), array([0.67106052, 0.60130719]), array([812, 711]))


#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.5732107682206172
(array([0.62193699, 0.51051051]), array([0.62048894, 0.51204819]), array([0.62121212, 0.5112782 ]), array([859, 664]))


#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(precision_recall_fscore_support(model.predict(X_test), y_test))

0.670387393302692
(array([0.78996499, 0.51651652]), array([0.67767768, 0.65648855]), array([0.72952586, 0.57815126]), array([999, 524]))
